Instalação e importações necessárias

In [1]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.5 MB/s eta 0:00:00


In [2]:
import os
os.environ['GROQ_API_KEY'] = "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"

Testando o funcionamento da API do Groq

In [6]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in various applications, offering numerous benefits that impact performance, efficiency, and user experience. Here are some reasons why fast language models are important:

1. **Real-time Interactions**: Fast language models enable real-time interactions in applications like virtual assistants, chatbots, and voice assistants. They can respond quickly to user queries, ensuring a seamless and engaging experience.
2. **Low Latency**: Fast language models reduce latency, which is critical in applications where timely responses are essential, such as in customer service, healthcare, or finance.
3. **Scalability**: Fast language models can handle a large volume of requests, making them suitable for large-scale applications, such as social media platforms, online forums, or language translation services.
4. **Energy Efficiency**: Fast language models can run on resource-constrained devices, such as smartphones or embedded systems, reducing energy consumption a

Criando a classe Agente

o método init recebe uma instância da API (cliente) e uma mensagem de sistema opcional, depois armazena ambos como atributos e cria uma lista para armazenar o histórico de mensagens

o método call recebe uma mensagem do usuário e adiciona ao histórico, chamando o método execute para obter uma resposta e, adiciona a resposta ao histórico de mensagem, retornando ela

por fim, o método execute envia o histórico de mensagens pra API do modelo, além de extrair e retornar o conteúdo da resposta gerada

In [7]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

Definindo o Prompt

vai definir o formato de interação do agente, estabelecendo um loop e descrevendo as duas funções disponíveis, a calculate para cálcuos matemáticos e get_planet_mass para obter a massa de um planeta

In [8]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

Definindo as funções do Agente

a calculte receberá uma string de uma operação matemática e através da função e val vai retornar o resultado

já a get_planet_mass receberá o nome de um planeta como entrada e retornar a sua massa em kg, mas através de notação científica

In [ ]:
def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

Execução do Agente

In [31]:
#cria uma instância da classe Agent, com o nome de neil_tyson
neil_tyson = Agent(client=client, system=system_prompt)

In [32]:
#envia uma pergunta ao agente, e ele responde com seu "pensamento inicial"
result = neil_tyson("what is the mass of Earth times 5?")
print(result)

Thought: I need to find the mass of Earth


In [34]:
#chama o agente novamente, mas agora ele decide executar a ação para obter a massa da Terra
result = neil_tyson()
print(result)

Action: get_planet_mass: Earth
PAUSE


In [35]:
#a função é chamada diretamente
result = get_planet_mass("Earth")
print(result)

5.972e+24


In [36]:
#envia o resultado da observação de volta ao agente, dai ele responde com o próximo pensamento
next_prompt = f"Observation: {result}"
result = neil_tyson(next_prompt)
print(result)

Thought: I need to multiply this by 5


In [38]:
#o agente decide executar a ação pra multiplicar a massa por 5 (seu ultimo pensamento)
result = neil_tyson()
print(result)

Action: calculate: 5.972e24 * 5
PAUSE


In [39]:
#chama a função pra calcular a operação matematica
observation = calculate("5.972e24 * 5")
print(observation)

2.9860000000000004e+25


In [40]:
#envia o resultado do cálculo de volta ao agente, com isso ele define que possui a resposta final
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Answer: The mass of Earth times 5 is 2.9860000000000004e+25.


In [41]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

Execução do Agente com loop

a função agent_loop automatiza o ciclo, recebendo as iterações, o prompt e a pergunta inicial

o loop principal incrementa o contador de iterações, envia o prompt atual ao agente e imprime sua resposta
se a resposta contém PAUSE ou ACTION, é extraído o nome da ferramenta e o argumento
se ele for válido, executa a função e prepara a observação

caso a resposta contenha ANSWER, encerra o loop, visto que está definido que seria a resposta final

In [42]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

In [43]:
agent_loop(10, system = system_prompt, query="what is the mass of the Earth plus the mass of Mercury, and All of it times 5")

Thought: I need to find the mass of the Earth and the mass of Mercury.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I have the mass of the Earth, now I need to find the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I have the mass of the Earth and Mercury, now I need to add them together.
Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: I have the sum of the masses of the Earth and Mercury, now I need to multiply it by 5.
Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Answer: The mass of the Earth plus the mass of Mercury, and all of it times 5 is 3.1502500000000004e+25.
